**1. Linear Models: Logestic regression**

**2. Non-linear models: Random forest, LightGBM, SVM**

In [ ]:
### Packages ######
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA




In [ ]:
#checking gender imbalance across drug conditions


# Load the dataset
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")

# Preview the column names to identify relevant ones
print(df.columns)

# Assuming the columns are named 'Gender' and 'DrugCondition'
# If they differ, replace with actual column names from the printout

# Count plot to visualize distribution
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='cond', hue='gender')
plt.title('Distribution of Drug Conditions Across Gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Crosstab for proportions
crosstab = pd.crosstab(df['cond'], df['gender'], normalize='index')
print("Proportion of Gender within each Drug Condition:")
print(crosstab)


**1.1. Logestic Regression (5-folds)**

In [ ]:
# ===  Load dataset ===
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
df = df.dropna(subset=["cond"])

# ===  Map education into 'low' and 'high' ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)

# ===  Drop unnecessary columns ===
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches"
]
X = df.drop(columns=drop_cols + ["cond"])
y = df["cond"]

# ===  Encode target ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# ===  Define feature groups ===
binary_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
scaled_cols = ["income", "age"]
categorical_cols = ["gender", "education", "kproto_cluster",
                    "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]

# ===  Preprocess features ===
preprocessor = ColumnTransformer([
    ("scale", StandardScaler(), scaled_cols),
    ("onehot", OneHotEncoder(drop="first", sparse_output=False), categorical_cols)
], remainder='passthrough')

X_processed = preprocessor.fit_transform(X)
cat_feature_names = preprocessor.named_transformers_["onehot"].get_feature_names_out(categorical_cols)
all_feature_names = list(cat_feature_names) + scaled_cols + binary_cols
X_processed_df = pd.DataFrame(X_processed, columns=all_feature_names)

# ===  Fit base logistic regression to get top 20 features ===
X_train_base, _, y_train_base, _ = train_test_split(
    X_processed_df, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

log_reg_base = LogisticRegression(max_iter=1000, multi_class="multinomial", solver="lbfgs")
log_reg_base.fit(X_train_base, y_train_base)
coef_importance = pd.Series(log_reg_base.coef_[0], index=all_feature_names)

# ===  Select top 20 features ===
top_20 = coef_importance.abs().sort_values(ascending=False).head(20).index.tolist()

# ===  Create valid interaction terms (only if both columns exist) ===
interaction_candidates = [
    ("kproto_cluster_1", "education_high"),
    ("kproto_cluster_2", "education_high"),
    ("pred_cluster_block1_1", "gender_2"),
    ("pred_cluster_block1_2", "gender_2")
]

valid_interactions = []
for col1, col2 in interaction_candidates:
    if col1 in X_processed_df.columns and col2 in X_processed_df.columns:
        new_col = f"{col1}__x__{col2}"
        X_processed_df[new_col] = X_processed_df[col1] * X_processed_df[col2]
        valid_interactions.append(new_col)

# ===  Combine top features and interactions ===
selected_features = top_20 + valid_interactions
X_selected = X_processed_df[selected_features]

# ===  Train/test split (stratified by gender) ===
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_encoded, test_size=0.2, stratify=X["gender"], random_state=42)

# ===  Fit and evaluate logistic regression ===
log_reg = LogisticRegression(max_iter=1000, multi_class="multinomial", solver="lbfgs")
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

# Evaluation
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Logistic Regression with Interactions")
plt.tight_layout()
plt.show()

# Cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_acc = cross_val_score(log_reg, X_train, y_train, cv=skf, scoring="accuracy")
cv_f1 = cross_val_score(log_reg, X_train, y_train, cv=skf, scoring="f1_macro")



# Print results
print(f"\nTest Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Test Macro F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
print(f"CV Accuracy (mean ± std): {cv_acc.mean():.3f} ± {cv_acc.std():.3f}")
print(f"CV Macro F1 (mean ± std): {cv_f1.mean():.3f} ± {cv_f1.std():.3f}")

################ visualizing the feature space to see how features are distirbuted
################ and why the model is unable to make any predictions on the class C############

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_selected)
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y_encoded, cmap='tab10', alpha=0.7)
plt.title("PCA of Selected Features")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(label="Class")
plt.tight_layout()
plt.show()


In [ ]:
# === Extract absolute coefficients for Logistic Regression feature importance ===
importance_series = pd.Series(
    log_reg.coef_[0], index=X_selected.columns
).abs().sort_values(ascending=False)

# === Display top 20 features ===
top_n = 20
top_features_lr = importance_series.head(top_n)

print(f"\nTop {top_n} Logistic Regression Most Important Features:\n")
print(top_features_lr)


In [ ]:


# === Get feature importance from trained model ===
feature_importance = pd.Series(
    np.abs(log_reg.coef_[0]),  # absolute value of coefficients
    index=X_selected.columns
).sort_values(ascending=True)

# === Plot ===
plt.figure(figsize=(12, 6))
feature_importance.plot(kind='barh', color='steelblue')
plt.title("Feature Importance - Logistic Regression")
plt.xlabel("Absolute Coefficient Value")
plt.tight_layout()
plt.show()


**2.1. Random Forest (3 and 5 folds)**

In [ ]:
# Load the dataset
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")

# ===  Map education and create interaction features ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)

# Create new interaction features
df['age_income_interaction'] = df['age'] * df['income']

# Convert boolean columns to integers
bool_cols_to_convert = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
for col in bool_cols_to_convert:
    df[col] = df[col].astype(int)

# ===  Drop unnecessary columns ===
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches"
]
X = df.drop(columns=drop_cols + ["cond"])
y = df["cond"]

# ===  Encode target ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# ===  Define feature groups (updated with new interaction feature) ===
scaled_cols = ["income", "age", "age_income_interaction"]
binary_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
categorical_cols = ["gender", "education", "kproto_cluster",
                    "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]

# Create a column transformer to apply different transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), scaled_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Create a pipeline with the preprocessor and the Random Forest model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# ===  Hyperparameter tuning using GridSearchCV ===
print("Performing hyperparameter tuning with GridSearchCV...")
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10, 20, None],
    'classifier__min_samples_leaf': [1, 2, 4]
}

cv_grid = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(model_pipeline, param_grid, cv=cv_grid, scoring='accuracy', n_jobs=-1, verbose=1)

grid_search.fit(X, y_encoded)

print("\nBest hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")


# Get the best model from the grid search
best_model = grid_search.best_estimator_

# === Define evaluation function ===
def evaluate_model(model, X, y, folds):
    print(f"\n=== {folds}-Fold Cross-Validation ===")
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    acc_scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    f1_scores = cross_val_score(model, X, y, cv=skf, scoring='f1_macro')

    for i, (acc, f1) in enumerate(zip(acc_scores, f1_scores), 1):
        print(f"Fold {i}: Accuracy = {acc:.4f}, Macro F1 = {f1:.4f}")

    # print(f"\nAverage Accuracy: {acc_scores.mean():.4f}")
    # print(f"Average Macro F1: {f1_scores.mean():.4f}")

      # Print results
        print(f"\nTest Accuracy: {accuracy_score(y_test, y_pred):.3f}")
        print(f"Test Macro F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
        print(f"CV Accuracy (mean ± std): {cv_acc.mean():.3f} ± {cv_acc.std():.3f}")
        print(f"CV Macro F1 (mean ± std): {cv_f1.mean():.3f} ± {cv_f1.std():.3f}")

# === Run evaluations ===
evaluate_model(best_model, X, y_encoded, folds=3)
evaluate_model(best_model, X, y_encoded, folds=5)
# ===  Plot feature importance with the best model and new features ===
print("\nPlotting feature importance with the best model...")

# Get feature names after preprocessing
ohe_feature_names = best_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_cols)
passthrough_feature_names = list(binary_cols)
preprocessed_feature_names = scaled_cols + list(ohe_feature_names) + passthrough_feature_names

# Get feature importances from the best Random Forest classifier
feature_importances = best_model.named_steps['classifier'].feature_importances_

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': preprocessed_feature_names,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the top 15 features
print("\nTop 15 Feature Importances with tuned model:")
print(importance_df.head(15))

# Plotting the feature importances
plt.figure(figsize=(12, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='steelblue')
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.title("Feature Importance - Random Forest")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig("tuned_feature_importance.png")
plt.show()

In [ ]:


# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# === Fit best model on training set ===
best_model.fit(X_train, y_train)

# === Predict on test set ===
y_pred = best_model.predict(X_test)

# === Classification Report ===
print("\n=== Classification Report (Random Forest) ===")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# === Confusion Matrix ===
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Random Forest (Test Set)")
plt.tight_layout()
plt.show()


**2.2. LightGBM (3 and 5 folds)**

In [ ]:

# === 1. Load dataset ===
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
df = df.dropna(subset=["cond"])

# === 2. Map education into 'low' and 'high' ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)

# === 3. Drop unnecessary columns ===
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches"
]
X = df.drop(columns=drop_cols + ["cond"])
y = df["cond"]

# === 4. Encode target ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# === 5. Define feature groups ===
scaled_cols = ["income", "age"]
binary_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
categorical_cols = ["gender", "education", "kproto_cluster",
                    "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]

# === 6. Preprocess features ===
preprocessor = ColumnTransformer([
    ("scale", StandardScaler(), scaled_cols),
    ("onehot", OneHotEncoder(drop="first", sparse_output=False), categorical_cols)
], remainder='passthrough')

X_processed = preprocessor.fit_transform(X)
cat_feature_names = preprocessor.named_transformers_["onehot"].get_feature_names_out(categorical_cols)
all_feature_names = list(cat_feature_names) + scaled_cols + binary_cols
X_processed_df = pd.DataFrame(X_processed, columns=all_feature_names)

# === 7. Add interaction terms ===
interaction_candidates = [
    ("kproto_cluster_1", "education_high"),
    ("kproto_cluster_2", "education_high"),
    ("pred_cluster_block1_1", "gender_2"),
    ("pred_cluster_block1_2", "gender_2"),
    ("income", "education_high"),
    ("age", "education_high")
]

for col1, col2 in interaction_candidates:
    if col1 in X_processed_df.columns and col2 in X_processed_df.columns:
        new_col = f"{col1}__x__{col2}"
        X_processed_df[new_col] = X_processed_df[col1] * X_processed_df[col2]

# === 8. Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_processed_df, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# === 9. Define and Tune LightGBM ===
print("\n=== Tuning LightGBM with GridSearchCV on Training Set ===")
model = LGBMClassifier(random_state=42, class_weight='balanced')

lgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=lgb_param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
val_score = grid_search.best_score_

print(f"\nBest LightGBM Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validated Macro F1 Score (Train CV): {val_score:.3f}")

# === 10. Define Evaluation Function ===
def evaluate_model(model, X, y, folds):
    print(f"\n=== {folds}-Fold Cross-Validation ===")
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)

    acc_scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    f1_scores = cross_val_score(model, X, y, cv=skf, scoring='f1_macro')

    for i, (acc, f1) in enumerate(zip(acc_scores, f1_scores), 1):
        print(f"Fold {i}: Accuracy = {acc:.4f}, Macro F1 = {f1:.4f}")

    avg_acc = acc_scores.mean()
    avg_f1 = f1_scores.mean()

    # print(f"\nAverage Accuracy: {avg_acc:.4f}")
    # print(f"Average Macro F1: {avg_f1:.4f}")

    ## Print results
    print(f"\nTest Accuracy: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Test Macro F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
    print(f"CV Accuracy (mean ± std): {cv_acc.mean():.3f} ± {cv_acc.std():.3f}")
    print(f"CV Macro F1 (mean ± std): {cv_f1.mean():.3f} ± {cv_f1.std():.3f}")

    return avg_acc, avg_f1

# === 11. Run Evaluations ===
acc_3, f1_3 = evaluate_model(best_model, X_processed_df, y_encoded, folds=3)
acc_5, f1_5 = evaluate_model(best_model, X_processed_df, y_encoded, folds=5)

# === 12. Evaluate on Held-Out Test Set ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\nTest Accuracy (Held-Out): {test_accuracy:.3f}")
print("\n=== Classification Report (Test Set) ===")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

# === 13. Confusion Matrix ===
cm_test = confusion_matrix(y_test, y_test_pred)
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=label_encoder.classes_)
disp_test.plot(cmap='Blues')
plt.title("Confusion Matrix - LightGBM (Test Set)")
plt.tight_layout()
plt.show()

# === 14. Feature Importance Plot ===
print("\n=== Feature Importances ===")
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
sns.barplot(x=importances[indices][:20], y=np.array(X_processed_df.columns)[indices][:20], palette="Blues")
plt.title("Feature Importances - LightGBM")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# === 15. Final Summary ===
print("\n=== Final Summary ===")
print(f"Validation Macro F1 (Train CV - GridSearch): {val_score:.3f}")
print(f"Test Accuracy (Held-Out): {test_accuracy:.3f}")
print(f"\nCross-Validation Results:")
print(f"  3-Fold CV: Accuracy = {acc_3:.3f}, Macro F1 = {f1_3:.3f}")
print(f"  5-Fold CV: Accuracy = {acc_5:.3f}, Macro F1 = {f1_5:.3f}")


In [ ]:
# === Print top 20 most important features ===
light_top_20_features = pd.DataFrame({
    "Feature": np.array(X_processed_df.columns)[indices][:20],
    "Importance": importances[indices][:20]
})
print("\nTop 20 Most Important Features:\n")
print(light_top_20_features)


**2.3. SVM**

In [ ]:
########### SVM 3-folds #############
# === 1. Load dataset ===
try:
    df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
except FileNotFoundError:
    print("Error: 'modeling_dataset_with_demo_and_age.csv' not found. Please ensure the file is in the correct directory.")
    # Exiting the script gracefully if the file is not found
    exit()

df = df.dropna(subset=["cond"])

# === 2. Split data before any processing to prevent data leakage ===
X = df.drop(columns=["cond"])
y = df["cond"]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# === 3. Map education and create interaction features ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
X_train["education"] = X_train["education"].replace(education_map)
X_test["education"] = X_test["education"].replace(education_map)

X_train['age_income_interaction'] = X_train['age'] * X_train['income']
X_test['age_income_interaction'] = X_test['age'] * X_test['income']

bool_cols_to_convert = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
for col in bool_cols_to_convert:
    X_train[col] = X_train[col].astype(int)
    X_test[col] = X_test[col].astype(int)

# === 4. Drop unnecessary columns ===
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches"
]
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

# === 5. Define feature groups ===
scaled_cols = ["income", "age", "age_income_interaction"]
categorical_cols = ["gender", "education", "kproto_cluster",
                    "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]

# === 6. Preprocess features and define SVM within a Pipeline ===
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), scaled_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop="first"), categorical_cols)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])

# === 7. Define and Tune SVM with an Expanded Grid ===
print("\n=== Tuning SVM with an expanded GridSearchCV on Training Set ===")
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100, 1000],
    'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'classifier__kernel': ['rbf']
}

cv_grid = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=param_grid,
    cv=cv_grid,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

# Fit GridSearchCV on the TRAINING data only
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
val_score = grid_search.best_score_

print(f"\n Best SVM Parameters: {grid_search.best_params_}")
print(f" Best Cross-Validated Macro F1 Score (Train CV): {val_score:.3f}")

# === 8. Evaluate on Held-Out Test Set ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\n Test Accuracy (Held-Out): {test_accuracy:.3f}")
print("\n=== Classification Report (Test Set) ===")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))


In [ ]:
################ SVM 5-folds #################

# === 1. Load dataset ===
try:
    df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
except FileNotFoundError:
    print("Error: 'modeling_dataset_with_demo_and_age.csv' not found. Please ensure the file is in the correct directory.")
    # Exiting the script gracefully if the file is not found
    exit()

df = df.dropna(subset=["cond"])

# === 2. Split data before any processing to prevent data leakage ===
X = df.drop(columns=["cond"])
y = df["cond"]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# === 3. Map education and create interaction features ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
X_train["education"] = X_train["education"].replace(education_map)
X_test["education"] = X_test["education"].replace(education_map)

X_train['age_income_interaction'] = X_train['age'] * X_train['income']
X_test['age_income_interaction'] = X_test['age'] * X_test['income']

bool_cols_to_convert = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
for col in bool_cols_to_convert:
    X_train[col] = X_train[col].astype(int)
    X_test[col] = X_test[col].astype(int)

# === 4. Drop unnecessary columns ===
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches"
]
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

# === 5. Define feature groups ===
scaled_cols = ["income", "age", "age_income_interaction"]
categorical_cols = ["gender", "education", "kproto_cluster",
                    "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]

# === 6. Preprocess features and define SVM within a Pipeline ===
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), scaled_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop="first"), categorical_cols)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])

# === 7. Define and Tune SVM with an Expanded Grid ===
print("\n=== Tuning SVM with an expanded GridSearchCV on Training Set ===")
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100, 1000],
    'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'classifier__kernel': ['rbf']
}

cv_grid = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=param_grid,
    cv=cv_grid,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

# Fit GridSearchCV on the TRAINING data only
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
val_score = grid_search.best_score_

print(f"\n Best SVM Parameters: {grid_search.best_params_}")
print(f"\n Cross-Validated Macro F1 Score (Train CV): {np.mean(val_score):.3f} ± {np.std(val_score):.3f}")

# === 8. Evaluate on Held-Out Test Set ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\n Test Accuracy (Held-Out): {test_accuracy:.3f}")
print("\n=== Classification Report (Test Set) ===")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))


In [ ]:
# === 1. Load and preprocess dataset ===
try:
    df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
except FileNotFoundError:
    print("Error: 'modeling_dataset_with_demo_and_age.csv' not found.")
    exit()

df = df.dropna(subset=["cond"])
X = df.drop(columns=["cond"])
y = df["cond"]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

# === 2. Feature engineering ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
X_train["education"] = X_train["education"].replace(education_map)
X_test["education"] = X_test["education"].replace(education_map)

X_train['age_income_interaction'] = X_train['age'] * X_train['income']
X_test['age_income_interaction'] = X_test['age'] * X_test['income']

bool_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
for col in bool_cols:
    X_train[col] = X_train[col].astype(int)
    X_test[col] = X_test[col].astype(int)

drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches"
]
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

scaled_cols = ["income", "age", "age_income_interaction"]
categorical_cols = ["gender", "education", "kproto_cluster",
                    "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]

# === 3. Define pipeline and parameter grid ===
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), scaled_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop="first"), categorical_cols)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])

param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100, 1000],
    'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'classifier__kernel': ['rbf']
}

# === 4. Run GridSearchCV with 3-fold and 5-fold ===
for folds in [3, 5]:
    print(f"\n=== Tuning SVM with {folds}-fold Cross-Validation ===")
    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        estimator=model_pipeline,
        param_grid=param_grid,
        cv=cv,
        scoring='f1_macro',
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    val_score = grid_search.best_score_
    print(f"\n Best Parameters ({folds}-fold): {grid_search.best_params_}")
    print(f" Best Macro F1 Score (Train CV): {val_score:.3f}")

    # === Evaluate on Test Set ===
    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"\n Test Accuracy ({folds}-fold model): {test_accuracy:.3f}")
    print("\n=== Classification Report (Test Set) ===")
    print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# === After each GridSearchCV ===
for folds in [3, 5]:
    print(f"\n=== Tuning SVM with {folds}-fold Cross-Validation ===")
    cv = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    grid_search = GridSearchCV(
        estimator=model_pipeline,
        param_grid=param_grid,
        cv=cv,
        scoring='f1_macro',
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    val_score = grid_search.best_score_
    cv_results = pd.DataFrame(grid_search.cv_results_)
    best_params = grid_search.best_params_

    mask = (
        (cv_results['param_classifier__C'] == best_params['classifier__C']) &
        (cv_results['param_classifier__gamma'] == best_params['classifier__gamma']) &
        (cv_results['param_classifier__kernel'] == best_params['classifier__kernel'])
    )

    cv_macro_f1_mean = cv_results.loc[mask, 'mean_test_score'].values[0]
    cv_macro_f1_std = cv_results.loc[mask, 'std_test_score'].values[0]

    acc_scores = cross_val_score(best_model, X_train, y_train, cv=cv, scoring='accuracy')
    cv_acc_mean = acc_scores.mean()
    cv_acc_std = acc_scores.std()

    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_macro_f1 = f1_score(y_test, y_test_pred, average='macro')

    print(f"\n=== SVM Performance Summary ({folds}-fold CV) ===")
    print(f"CV Accuracy (Mean ± Std): {cv_acc_mean:.3f} ± {cv_acc_std:.3f}")
    print(f"CV Macro F1 (Mean ± Std): {cv_macro_f1_mean:.3f} ± {cv_macro_f1_std:.3f}")
    print(f"Test Accuracy: {test_accuracy:.3f}")
    print(f"Test Macro F1: {test_macro_f1:.3f}")


In [ ]:
# === Generate Confusion Matrix ===
cm = confusion_matrix(y_test, y_test_pred)

# === Plot Confusion Matrix as Heatmap ===
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix - SVM Model')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

# === Manually transform X_test using the preprocessor ===
X_test_transformed = best_model.named_steps['preprocessor'].transform(X_test)

# === Apply permutation importance to the classifier only ===
perm_result = permutation_importance(
    best_model.named_steps['classifier'], X_test_transformed, y_test,
    n_repeats=10, scoring='f1_macro', random_state=42, n_jobs=-1
)

# === Get correct feature names from preprocessor ===
feature_names = best_model.named_steps['preprocessor'].get_feature_names_out()

# === Create DataFrame of importances ===
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_result.importances_mean
}).sort_values(by='Importance', ascending=False)

# === Display top 20 features ===
print("\n Top 20 Most Important Features (SVM - Permutation Importance):\n")
print(importance_df.head(20))


In [ ]:
import matplotlib.pyplot as plt

# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

# Plot 1
axs[0, 0].plot(x1, y1)
axs[0, 0].set_title('Plot 1')

# Plot 2
axs[0, 1].plot(x2, y2)
axs[0, 1].set_title('Plot 2')

# Plot 3
axs[1, 0].plot(x3, y3)
axs[1, 0].set_title('Plot 3')

# Plot 4
axs[1, 1].plot(x4, y4)
axs[1, 1].set_title('Plot 4')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:


# === Select top 20 features ===
top_features = importance_df.head(20)

# === Set plot style ===
sns.set(style="whitegrid")

# === Create horizontal bar plot ===
plt.figure(figsize=(12, 6))
sns.barplot(
    x="Importance", y="Feature",
    data=top_features,
    palette="Blues"
)

# === Add plot labels and title ===
plt.title("Feature Importance - SVM", fontsize=14)
plt.xlabel("Mean Importance", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.tight_layout()

# === Show plot ===
plt.show()


**Model Comparision on general feature set (5-fold)**

In [ ]:
###### most consistant features among all models that consistently appeared with notable importance in at least three out of four models

general_features = [
    "switched_block3",
    "switched_1_to_2",
    "switched_2_to_3",
    "pred_cluster_block3_1",
    "pred_cluster_block3_2",
    "pred_cluster_block1_2",
    "pred_cluster_block1_1",
    "pred_cluster_block2_1",
    "kproto_cluster_1",
    "kproto_cluster_2",
    "age_income_interaction",
    "age",
    "income",
    "education_low",
    "gender_2",
    "gender_3"
]



In [ ]:
######### Logistic Regression ##############

# === 1. Load and preprocess dataset ===
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
df = df.dropna(subset=["cond"])

# Map education
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)
df["education_low"] = (df["education"] == "low").astype(int)

# Create interaction term
df["age_income_interaction"] = df["age"] * df["income"]

# One-hot encode gender
df["gender_2"] = (df["gender"] == 2).astype(int)
df["gender_3"] = (df["gender"] == 3).astype(int)

# One-hot encode cluster features
cluster_cols = ["kproto_cluster", "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]
df = pd.get_dummies(df, columns=cluster_cols, prefix=cluster_cols, drop_first=False)

# Convert boolean columns to int
bool_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
df[bool_cols] = df[bool_cols].astype(int)

# Encode target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["cond"])

# Drop unused columns
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches", "cond"
]
df = df.drop(columns=drop_cols)

# === 2. Select general feature set ===
general_features = [
    "switched_block3", "switched_1_to_2", "switched_2_to_3",
    "pred_cluster_block3_1", "pred_cluster_block3_2",
    "pred_cluster_block1_2", "pred_cluster_block1_1",
    "pred_cluster_block2_1", "kproto_cluster_1", "kproto_cluster_2",
    "age_income_interaction", "age", "income",
    "education_low", "gender_2", "gender_3"
]

X_general = df[general_features]

# === 3. Scale numeric features ===
scaler = StandardScaler()
scaled_cols = ["income", "age", "age_income_interaction"]
X_general[scaled_cols] = scaler.fit_transform(X_general[scaled_cols])

# === 4. Train/test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_general, y_encoded, test_size=0.2, stratify=df["gender"], random_state=42
)

# === 5. Fit logistic regression ===
log_reg = LogisticRegression(max_iter=1000, multi_class="multinomial", solver="lbfgs")
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

# === 6. Evaluation ===
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Logistic Regression (General Features)")
plt.tight_layout()
plt.show()

# === 7. Cross-validation ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_acc = cross_val_score(log_reg, X_train, y_train, cv=skf, scoring="accuracy")
cv_f1 = cross_val_score(log_reg, X_train, y_train, cv=skf, scoring="f1_macro")

print(f"\nTest Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Test Macro F1: {f1_score(y_test, y_pred, average='macro'):.3f}")
print(f"CV Accuracy (mean ± std): {cv_acc.mean():.3f} ± {cv_acc.std():.3f}")
print(f"CV Macro F1 (mean ± std): {cv_f1.mean():.3f} ± {cv_f1.std():.3f}")

# === PCA Visualization with Class Labels ===
import matplotlib.patches as mpatches

# Map encoded labels back to original class names
class_names = label_encoder.classes_
y_labels = [class_names[i] for i in y_encoded]

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# === Apply PCA to reduce general feature matrix to 2 components ===
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_general)

# === Map encoded labels back to original class names ===
class_names = label_encoder.classes_  # ['c', 'h', 'p', 'y']
label_map = dict(enumerate(class_names))
y_labels = [label_map[i] for i in y_encoded]

# === Create a DataFrame for PCA scatter plot ===
lr_pca_df = pd.DataFrame({
    "PC1": X_pca[:, 0],
    "PC2": X_pca[:, 1],
    "Class": y_labels
})

# === Plot PCA scatter with class labels ===
plt.figure(figsize=(12, 6))
sns.scatterplot(
    data=lr_pca_df,
    x="PC1", y="PC2",
    hue="Class",
    palette="Set2",
    alpha=0.7
)
plt.title("PCA Visualization of General Feature Set (Logistic Regression)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Class")
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:


# # Apply PCA to reduce general feature matrix to 2 components
# pca = PCA(n_components=2)
# X_pca = pca.fit_transform(X_general)

# # Map encoded labels back to original class names
# class_names = label_encoder.classes_  # ['c', 'h', 'p', 'y'] — adjust if needed
# label_map = dict(enumerate(class_names))
# y_labels = [label_map[i] for i in y_encoded]

# # Create a DataFrame for PCA scatter plot
# lr_pca_df = pd.DataFrame({
#     "PC1": X_pca[:, 0],
#     "PC2": X_pca[:, 1],
#     "Class": y_labels
# })

# # Plot PCA scatter with class labels
# plt.figure(figsize=(12, 6))
# sns.scatterplot(
#     data=lr_pca_df,
#     x="PC1", y="PC2",
#     hue="Class",
#     palette="Set2",
#     alpha=0.7
# )
# plt.title("PCA Visualization of General Feature Set (Logistic Regression)")
# plt.xlabel("Principal Component 1")
# plt.ylabel("Principal Component 2")
# plt.legend(title="Class")
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
################# Random Forest ########################


# === 1. Load dataset ===
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
df = df.dropna(subset=["cond"])

# === 2. Encode target ===
y = df["cond"]
label_encoder = LabelEncoder()
fr_y_encoded = label_encoder.fit_transform(y)

# === 3. Feature engineering ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)
df["education_low"] = (df["education"] == "low").astype(int)
df["age_income_interaction"] = df["age"] * df["income"]
df["gender_2"] = (df["gender"] == 2).astype(int)
df["gender_3"] = (df["gender"] == 3).astype(int)

# One-hot encode cluster features
cluster_cols = ["kproto_cluster", "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]
df = pd.get_dummies(df, columns=cluster_cols, prefix=cluster_cols, drop_first=False)

# Convert boolean columns to int
bool_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
df[bool_cols] = df[bool_cols].astype(int)

# Drop unnecessary columns
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches", "cond"
]
df = df.drop(columns=drop_cols)

# === 4. Select general feature set ===
general_features = [
    "switched_block3", "switched_1_to_2", "switched_2_to_3",
    "pred_cluster_block3_1", "pred_cluster_block3_2",
    "pred_cluster_block1_2", "pred_cluster_block1_1",
    "pred_cluster_block2_1", "kproto_cluster_1", "kproto_cluster_2",
    "age_income_interaction", "age", "income",
    "education_low", "gender_2", "gender_3"
]

X = df[general_features]

# === 5. Preprocessing and pipeline ===
scaled_cols = ["income", "age", "age_income_interaction"]
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), scaled_cols)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# === 6. Hyperparameter tuning ===
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10, 20, None],
    'classifier__min_samples_leaf': [1, 2, 4]
}

cv_grid = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rf_grid_search = GridSearchCV(model_pipeline, param_grid, cv=cv_grid, scoring='f1_macro', n_jobs=-1, verbose=1)
rf_grid_search.fit(X, y_encoded)

rf_best_model = grid_search.best_estimator_
rf_val_score = grid_search.best_score_

print("\nBest Random Forest Parameters:")
print(grid_search.best_params_)
print(f"Best CV Macro F1 Score (Train CV): {rf_val_score:.3f}")

# === 7. 5-Fold Evaluation ===
rf_acc_scores = cross_val_score(best_model, X, fr_y_encoded, cv=cv_grid, scoring='accuracy')
rf_f1_scores = cross_val_score(best_model, X, fr_y_encoded, cv=cv_grid, scoring='f1_macro')

print("\n=== Random Forest Performance (5-Fold CV) ===")
for i, (acc, f1) in enumerate(zip(rf_acc_scores, rf_f1_scores), 1):
    print(f"Fold {i}: Accuracy = {acc:.4f}, Macro F1 = {f1:.4f}")

print(f"\nCV Accuracy (mean ± std): {rf_acc_scores.mean():.3f} ± {rf_acc_scores.std():.3f}")
print(f"CV Macro F1 (mean ± std): {rf_f1_scores.mean():.3f} ± {rf_f1_scores.std():.3f}")



In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Apply PCA to reduce to 2 components
pca = PCA(n_components=2)
rf_X_pca = pca.fit_transform(X)

# Map encoded labels back to class names
class_names = label_encoder.inverse_transform([0, 1, 2, 3])  # Adjust if needed
label_map = dict(zip(range(len(class_names)), class_names))
fr_y_labels = [label_map[i] for i in fr_y_encoded]

# Create a DataFrame for plotting
rf_pca_df = pd.DataFrame({
    "PC1": rf_X_pca[:, 0],
    "PC2": rf_X_pca[:, 1],
    "Class": fr_y_labels
})

# Plot
plt.figure(figsize=(12, 6))
sns.scatterplot(data=rf_pca_df, x="PC1", y="PC2", hue="Class", palette="Set2", alpha=0.7)
plt.title("PCA Visualization of General Feature Set (Random Forest)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Class")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# === 10. Report results ===
print("\n=== Random Forest Performance Summary ===")
print(f"CV Accuracy (Mean ± Std): {rf_acc_scores.mean():.3f} ± {rf_acc_scores.std():.3f}")
print(f"CV Macro F1 (Mean ± Std): {rf_f1_scores.mean():.3f} ± {rf_f1_scores.std():.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Test Macro F1: {test_macro_f1:.3f}")

In [ ]:
################# LightGBM #######################


# === 1. Load dataset ===
df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
df = df.dropna(subset=["cond"])

# === 2. Encode target ===
y = df["cond"]
label_encoder = LabelEncoder()
light_y_encoded = label_encoder.fit_transform(y)

# === 3. Feature engineering ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)
df["education_low"] = (df["education"] == "low").astype(int)
df["age_income_interaction"] = df["age"] * df["income"]
df["gender_2"] = (df["gender"] == 2).astype(int)
df["gender_3"] = (df["gender"] == 3).astype(int)

# One-hot encode cluster features
cluster_cols = ["kproto_cluster", "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]
df = pd.get_dummies(df, columns=cluster_cols, prefix=cluster_cols, drop_first=False)

# Convert boolean columns to int
bool_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
df[bool_cols] = df[bool_cols].astype(int)

# Drop unnecessary columns
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches", "cond"
]
df = df.drop(columns=drop_cols)

# === 4. Select general feature set ===
general_features = [
    "switched_block3", "switched_1_to_2", "switched_2_to_3",
    "pred_cluster_block3_1", "pred_cluster_block3_2",
    "pred_cluster_block1_2", "pred_cluster_block1_1",
    "pred_cluster_block2_1", "kproto_cluster_1", "kproto_cluster_2",
    "age_income_interaction", "age", "income",
    "education_low", "gender_2", "gender_3"
]

X = df[general_features]

# === 5. Define LightGBM model ===
light_model = LGBMClassifier(random_state=42, class_weight='balanced')

# === 6. 5-Fold Cross-Validation ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

light_acc_scores = cross_val_score(light_model, X, light_y_encoded, cv=cv, scoring='accuracy')
light_f1_scores = cross_val_score(light_model, X, light_y_encoded, cv=cv, scoring='f1_macro')

# === 7. Report results ===
print("\n=== LightGBM Performance (5-Fold CV) ===")
for i, (acc, f1) in enumerate(zip(light_acc_scores, light_f1_scores), 1):
    print(f"Fold {i}: Accuracy = {acc:.4f}, Macro F1 = {f1:.4f}")

print(f"\nCV Accuracy (mean ± std): {light_acc_scores.mean():.3f} ± {light_acc_scores.std():.3f}")
print(f"CV Macro F1 (mean ± std): {light_f1_scores.mean():.3f} ± {light_f1_scores.std():.3f}")


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# === 1. Apply PCA ===
light_pca = PCA(n_components=2)
light_X_pca = pca.fit_transform(X)

# === 2. Map encoded labels to class names ===
class_names = label_encoder.inverse_transform([0, 1, 2, 3])  # Adjust if needed
label_map = dict(zip(range(len(class_names)), class_names))
light_y_labels = [label_map[i] for i in light_y_encoded]

# === 3. Create DataFrame for plotting ===
light_pca_df = pd.DataFrame({
    "PC1": light_X_pca[:, 0],
    "PC2": light_X_pca[:, 1],
    "Class": light_y_labels
})

# === 4. Plot PCA scatter ===
plt.figure(figsize=(12, 6))
sns.scatterplot(data=light_pca_df, x="PC1", y="PC2",  hue="Class", palette="Set2", alpha=0.7)
plt.title("PCA Visualization of General Feature Set (LightGBM)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Class")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# === 8. Fit and evaluate on test set ===
light_model.fit(X_train, y_train)
light_y_test_pred = model.predict(X_test)
light_test_accuracy = accuracy_score(y_test, light_y_test_pred)
light_test_macro_f1 = f1_score(y_test, light_y_test_pred, average='macro')

# === 9. Report results ===
print("\n=== LightGBM Performance Summary ===")
print(f"CV Accuracy (Mean ± Std): {light_acc_scores.mean():.3f} ± {light_acc_scores.std():.3f}")
print(f"CV Macro F1 (Mean ± Std): {light_f1_scores.mean():.3f} ± {light_f1_scores.std():.3f}")
print(f"Test Accuracy: {light_test_accuracy:.3f}")
print(f"Test Macro F1: {light_test_macro_f1:.3f}")

In [ ]:
################ SVM #####################


# === 1. Load dataset ===
try:
    df = pd.read_csv("modeling_dataset_with_demo_and_age.csv")
except FileNotFoundError:
    print("Error: 'modeling_dataset_with_demo_and_age.csv' not found.")
    exit()

df = df.dropna(subset=["cond"])

# === 2. Encode target ===
y = df["cond"]
label_encoder = LabelEncoder()
svm_y_encoded = label_encoder.fit_transform(y)

# === 3. Map education and create interaction features ===
education_map = {1: "low", 2: "low", 3: "low", 4: "high", 5: "high"}
df["education"] = df["education"].replace(education_map)
df["education_low"] = (df["education"] == "low").astype(int)
df["age_income_interaction"] = df["age"] * df["income"]

# === 4. One-hot encode gender ===
df["gender_2"] = (df["gender"] == 2).astype(int)
df["gender_3"] = (df["gender"] == 3).astype(int)

# === 5. One-hot encode cluster features ===
cluster_cols = ["kproto_cluster", "pred_cluster_block1", "pred_cluster_block2", "pred_cluster_block3"]
df = pd.get_dummies(df, columns=cluster_cols, prefix=cluster_cols, drop_first=False)

# === 6. Convert boolean columns to int ===
bool_cols = ["switched_block1", "switched_block2", "switched_block3", "switched_1_to_2", "switched_2_to_3"]
df[bool_cols] = df[bool_cols].astype(int)

# === 7. Drop unnecessary columns ===
drop_cols = [
    "subj_num", "transition_block1", "transition_block2", "transition_block3",
    "transition_1_to_2", "transition_2_to_3", "total_switches", "sequential_switches", "cond"
]
df = df.drop(columns=drop_cols)

# === 8. Define general feature set ===
general_features = [
    "switched_block3", "switched_1_to_2", "switched_2_to_3",
    "pred_cluster_block3_1", "pred_cluster_block3_2",
    "pred_cluster_block1_2", "pred_cluster_block1_1",
    "pred_cluster_block2_1", "kproto_cluster_1", "kproto_cluster_2",
    "age_income_interaction", "age", "income",
    "education_low", "gender_2", "gender_3"
]

X = df[general_features]

# === 9. Train/test split ===
svm_X_train, svm_X_test, svm_y_train, y_test = train_test_split(
    X, svm_y_encoded, test_size=0.2, stratify=df["gender"], random_state=42
)

# === 10. Preprocessing and SVM pipeline ===
scaled_cols = ["income", "age", "age_income_interaction"]
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), scaled_cols)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(random_state=42))
])

# === 11. Grid search for hyperparameter tuning ===
print("\n=== Tuning SVM with GridSearchCV ===")
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100, 1000],
    'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'classifier__kernel': ['rbf']
}

cv_grid = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=model_pipeline,
    param_grid=param_grid,
    cv=cv_grid,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
val_score = grid_search.best_score_

print(f"\nBest SVM Parameters: {grid_search.best_params_}")
print(f"Best CV Macro F1 Score (Train CV): {val_score:.3f}")

# === 12. Cross-validation std for best config ===
cv_results = pd.DataFrame(grid_search.cv_results_)
best_params = grid_search.best_params_

mask = (
    (cv_results['param_classifier__C'] == best_params['classifier__C']) &
    (cv_results['param_classifier__gamma'] == best_params['classifier__gamma']) &
    (cv_results['param_classifier__kernel'] == best_params['classifier__kernel'])
)

best_cv_scores = cv_results.loc[mask, 'mean_test_score']
best_cv_std = cv_results.loc[mask, 'std_test_score']

print(f"CV Macro F1 (mean ± std): {best_cv_scores.values[0]:.3f} ± {best_cv_std.values[0]:.3f}")

# === 13. Evaluate on test set ===
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\nTest Accuracy (Held-Out): {test_accuracy:.3f}")
print("\n=== Classification Report (Test Set) ===")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# === 1. Apply PCA ===
pca = PCA(n_components=2)
svm_X_pca = pca.fit_transform(X)

# === 2. Map encoded labels to class names ===
class_names = label_encoder.inverse_transform([0, 1, 2, 3])  # Adjust if needed
label_map = dict(zip(range(len(class_names)), class_names))
y_labels = [label_map[i] for i in y_encoded]

# === 3. Create DataFrame for plotting ===
svm_pca_df = pd.DataFrame({
    "PC1": X_pca[:, 0],
    "PC2": X_pca[:, 1],
    "Class": y_labels
})

# === 4. Plot PCA scatter ===
plt.figure(figsize=(12, 6))
sns.scatterplot(data=svm_pca_df, x="PC1", y="PC2", hue="Class", palette="Set2", alpha=0.7)
plt.title("PCA Visualization of General Feature Set (SVM)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Class")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Skip re-creating PCA DataFrames — use them directly
dfs = [lr_pca_df, rf_pca_df, light_pca_df, svm_pca_df]
titles = ["Logistic Regression", "Random Forest", "LightGBM", "SVM"]

# Plot all in a 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, df, title in zip(axes.flatten(), dfs, titles):
    sns.scatterplot(data=df, x="PC1", y="PC2", hue="Class", palette="Set1", alpha=0.7, ax=ax)
    ax.set_title(f"PCA Visualization ({title})")
    ax.set_xlabel("Principal Component 1")
    ax.set_ylabel("Principal Component 2")
    ax.legend(title="Class", loc='best')
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:

# === CV Accuracy (Mean ± Std) ===
cv_accuracy_mean = cv_results.loc[mask, 'mean_test_score'].values[0]
cv_accuracy_std = cv_results.loc[mask, 'std_test_score'].values[0]

# === CV Macro F1 (Mean ± Std) ===
cv_macro_f1_mean = cv_results.loc[mask, 'mean_test_score'].values[0]
cv_macro_f1_std = cv_results.loc[mask, 'std_test_score'].values[0]

# === Test Accuracy ===
test_accuracy = accuracy_score(y_test, y_test_pred)

# === Test Macro F1 ===
test_macro_f1 = f1_score(y_test, y_test_pred, average='macro')

# === Print all metrics ===
print("\n=== SVM Performance Summary ===")
print(f"CV Accuracy (Mean ± Std): {cv_accuracy_mean:.3f} ± {cv_accuracy_std:.3f}")
print(f"CV Macro F1 (Mean ± Std): {cv_macro_f1_mean:.3f} ± {cv_macro_f1_std:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"Test Macro F1: {test_macro_f1:.3f}")


**Algo-specific Model Comparision, 3-folds**

In [ ]:

models = {
    "SVM": SVC(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}


param_grids = {
    "SVM": {
        'classifier__C': [0.1, 1, 10],
        'classifier__gamma': [0.01, 0.1],
        'classifier__kernel': ['rbf']
    },
    "RandomForest": {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [5, 10]
    },
    "LogisticRegression": {
        'classifier__C': [0.1, 1, 10]
    },
    "LightGBM": {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [3, 5],
        'classifier__learning_rate': [0.01, 0.1]
    }
}

from sklearn.pipeline import Pipeline

results = []

for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    grid = param_grids[name]
    grid_search = GridSearchCV(pipeline, grid, cv=3, scoring='f1_macro', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    results.append({
        "Model": name,
        "Best Params": grid_search.best_params_,
        "CV Macro F1": grid_search.best_score_,
        "Test Accuracy": accuracy_score(y_test, y_pred),
        "Test Macro F1": f1_score(y_test, y_pred, average='macro')
    })

results_df = pd.DataFrame(results)
print("\n=== Model Comparison ===")
print(results_df.sort_values(by="Test Macro F1", ascending=False))


**Algo-specific Model Comparision, 5-folds**

In [ ]:

models = {
    "SVM": SVC(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}


param_grids = {
    "SVM": {
        'classifier__C': [0.1, 1, 10],
        'classifier__gamma': [0.01, 0.1],
        'classifier__kernel': ['rbf']
    },
    "RandomForest": {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [5, 10]
    },
    "LogisticRegression": {
        'classifier__C': [0.1, 1, 10]
    },
    "LightGBM": {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [3, 5],
        'classifier__learning_rate': [0.01, 0.1]
    }
}

from sklearn.pipeline import Pipeline

results = []

for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    grid = param_grids[name]
    grid_search = GridSearchCV(pipeline, grid, cv=5, scoring='f1_macro', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    results.append({
        "Model": name,
        "Best Params": grid_search.best_params_,
        "CV Macro F1": grid_search.best_score_,
        "Test Accuracy": accuracy_score(y_test, y_pred),
        "Test Macro F1": f1_score(y_test, y_pred, average='macro')
    })

results_df = pd.DataFrame(results)
print("\n=== Model Comparison ===")
print(results_df.sort_values(by="Test Macro F1", ascending=False))


**Testing model performance (best model, LightGBM) for gender**

In [ ]:
gender_test = X.loc[X_test.index, "gender"]
from sklearn.metrics import classification_report

print("\n=== Gender-Based Performance ===")
for gender_value in sorted(gender_test.unique()):
    mask = gender_test == gender_value
    y_true_gender = y_test[mask]
    y_pred_gender = y_test_pred[mask]

    print(f"\n--- Gender: {gender_value} ---")
    print(f"Support: {len(y_true_gender)}")
    print(classification_report(y_true_gender, y_pred_gender, target_names=label_encoder.classes_))
